In [ ]:
import pandas as pd
import openpyxl
import duckdb

Setup database

In [ ]:
conn = duckdb.connect('returns.db')
conn.execute("INSTALL 'excel'")
conn.execute("LOAD 'excel'")
table = 'returns'
file = 'RETRO_SAMPLE.xlsx'
conn.execute("CREATE TABLE returns AS SELECT * FROM RETRO_SAMPLE.xlsx")

DB information

In [ ]:
tables = conn.execute("SHOW TABLES").fetchall()
print(tables)
columns = conn.execute("PRAGMA table_info('returns');").fetchall()
print(columns)
query = conn.execute("""
    SELECT COUNT(DISTINCT CUSTOMER_EMAILID)
    FROM returns
""").fetchone()

print(f"unqiue customer emails: {query}")

cust_orders = conn.execute("""
    SELECT AVG(order_count) AS avg_unique_orders_per_customer
    FROM (
        SELECT CUSTOMER_EMAILID, COUNT(DISTINCT SALES_ORDER_NO) AS order_count
        FROM returns
        GROUP BY CUSTOMER_EMAILID
    );
        """).fetchone()

print(f"average unique orders per customer: {cust_orders}")

cust_returns = conn.execute("""
    SELECT AVG(return_count) AS avg_unique_returns_per_customer
    FROM (
        SELECT CUSTOMER_EMAILID, COUNT(DISTINCT RETURN_NO) AS return_count
        FROM returns
        GROUP BY CUSTOMER_EMAILID
    );
        """).fetchone()

print(f"average unique returns per customer: {cust_returns}")